#  Big Data Management Assignment 1

Importing all sparkcontext, sparkconf,sparksession from pyspark 

In [3]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession

In [4]:
sc = SparkContext.getOrCreate();

In [5]:
spark=SparkSession.builder.getOrCreate()

In [6]:
ss= spark

### 1) Loading the csv file from the Hdfs file system

Loading the csv file from hdfs requires the hadoop services to be started.

In [12]:
vehicle_records_DF = spark.read.csv("hdfs://localhost:9000/user/lalith/data/per-vehicle-records-2021-01-31.csv", inferSchema = True, header = True)


A DataFrame vehicle_records_DF has been created with default schema.

In [89]:
vehicle_records_DF.show()

+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+
|cosit|year|month|day|hour|minute|second|millisecond|minuteofday|lane|lanename|straddlelane|straddlelanename|class|classname|length|headway| gap|speed|weight|temperature|duration|validitycode|numberofaxles|axleweights|axlespacings|
+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+
|  998|2021|    1| 31|   2|    45|     0|          0|        165|   2|    Ch 2|           0|            null|    2|      CAR|   5.2|   1.07|1.13| 71.0|   0.0|        0.0|       0|           0|            0|       null|        null|
|  998|2021|    1| 31|   2|    45|     1|          0|        165|   2|  

Count of number of records in the loaded csv file.

In [107]:
vehicle_records_DF.count()

1106652

In [91]:
vehicle_records_DF.schema

StructType(List(StructField(cosit,IntegerType,true),StructField(year,IntegerType,true),StructField(month,IntegerType,true),StructField(day,IntegerType,true),StructField(hour,IntegerType,true),StructField(minute,IntegerType,true),StructField(second,IntegerType,true),StructField(millisecond,IntegerType,true),StructField(minuteofday,IntegerType,true),StructField(lane,IntegerType,true),StructField(lanename,StringType,true),StructField(straddlelane,IntegerType,true),StructField(straddlelanename,StringType,true),StructField(class,IntegerType,true),StructField(classname,StringType,true),StructField(length,DoubleType,true),StructField(headway,DoubleType,true),StructField(gap,DoubleType,true),StructField(speed,DoubleType,true),StructField(weight,DoubleType,true),StructField(temperature,DoubleType,true),StructField(duration,IntegerType,true),StructField(validitycode,IntegerType,true),StructField(numberofaxles,IntegerType,true),StructField(axleweights,StringType,true),StructField(axlespacings,Str

In [20]:
type(vehicle_records_DF)

pyspark.sql.dataframe.DataFrame

Before performing the analysis on the Dataset, Intially we need to filter the dataset by identifying the M50 cosits from  junction 3 to junction 17 as per requirement.

Firstly, we need to identify the cosits on m50 highway by refering to the map of irish road traffic data and the dataset.
There are 15 counters present on m50 highway out of which 13 counters present in between junction 3 to 17. 

we need data for only those counters. so let's create a dataframe that contains only the required data by filtering the orginial dataset. 

In [25]:
cositlist = [1500,1501,1502,1508,1503,1509,1504,1505,1506,1507,15010,15011,15012]

m50cositdf = vehicle_records_DF.where(vehicle_records_DF.cosit.isin(cositlist))

In [26]:
type(m50cositdf)

pyspark.sql.dataframe.DataFrame

In [ ]:
The below dataframe consists of records for only m50 counters between junction 3 to junction 17

In [27]:
m50cositdf.show()

+-----+----+-----+---+----+------+------+-----------+-----------+----+------------+------------+----------------+-----+---------+------+-------+------+-----+------+-----------+--------+------------+-------------+-----------+------------+
|cosit|year|month|day|hour|minute|second|millisecond|minuteofday|lane|    lanename|straddlelane|straddlelanename|class|classname|length|headway|   gap|speed|weight|temperature|duration|validitycode|numberofaxles|axleweights|axlespacings|
+-----+----+-----+---+----+------+------+-----------+-----------+----+------------+------------+----------------+-----+---------+------+-------+------+-----+------+-----------+--------+------------+-------------+-----------+------------+
| 1500|2021|    1| 31|   2|    45|     7|          0|        165|   2|Northbound 2|           0|            null|    3|      LGV|   5.1|  233.7|233.53|101.0|   0.0|        0.0|       0|           0|            0|       null|        null|
| 1500|2021|    1| 31|   2|    45|    16|       

In [28]:
m50cositdf.select("cosit").count()

181582

#### Calculating the usage of Irish road network in terms of percentage by vehicle category

In [30]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col

In [106]:
m50cositdf.groupby('classname').agg(
    (F.count('classname')).alias('count'),
    (F.round(F.count('classname') / m50cositdf.count() * 100,3)).alias('percentage')
).show()

+---------+------+----------+
|classname| count|percentage|
+---------+------+----------+
|      CAR|154927|    85.321|
|  HGV_ART|  5460|     3.007|
|      BUS|  1107|      0.61|
|  HGV_RIG|  1262|     0.695|
|     null|     0|       0.0|
|  CARAVAN|   610|     0.336|
|      LGV| 17860|     9.836|
|    MBIKE|   355|     0.196|
+---------+------+----------+



From the above query we can find the usage of irish road network by each vehicle class. From the result we can observe that classname CAR has the highest uasge with percentage of 85.321

#### Calculating the highest and lowest hourly fows on M50 - show the hours and total number of vehicle counts.

In [42]:
m50cositdf.select("hour").groupBy("hour").count().sort("count", ascending=False).show(24)

+----+-----+
|hour|count|
+----+-----+
|  15|17211|
|  16|16575|
|  14|16373|
|  13|15784|
|  17|14742|
|  12|11768|
|  18|11503|
|  11|10943|
|  19| 9769|
|  20| 9437|
|  10| 8310|
|  21| 6426|
|   9| 6055|
|   7| 5720|
|   8| 4922|
|   6| 3483|
|  22| 3389|
|  23| 2664|
|   0| 1844|
|   5| 1288|
|   1| 1187|
|   2|  891|
|   4|  788|
|   3|  510|
+----+-----+



The above query gives the count of vehicles per hour, we can observe that hours 13 -17 has highest uage, while hours 1 - 5 has least uasge

#### Calculating the evening and morning rush hours on M50 - show the hours and the total counts

In [43]:
morning_hours = [9,10,11]
evening_hours = [20,21,22]

In [47]:
m50cositdf.where(m50cositdf.hour.isin(morning_hours)).select(m50cositdf.hour.alias("Morning rush hours between 9-12")).groupBy("Morning rush hours between 9-12").count().sort("Morning rush hours between 9-12", ascending= False).show()

+-------------------------------+-----+
|Morning rush hours between 9-12|count|
+-------------------------------+-----+
|                             11|10943|
|                             10| 8310|
|                              9| 6055|
+-------------------------------+-----+



The above query identifies the morning rush hours and the count of vehicles per that hour. We can observe that morning rush hours are between 9 -12

In [49]:
m50cositdf.where(m50cositdf.hour.isin(evening_hours)).select(m50cositdf.hour.alias("Evening rush hours between 20-22")).groupBy("Evening rush hours between 20-22").count().sort("Evening rush hours between 20-22", ascending= False).show()

+--------------------------------+-----+
|Evening rush hours between 20-22|count|
+--------------------------------+-----+
|                              22| 3389|
|                              21| 6426|
|                              20| 9437|
+--------------------------------+-----+



The above query identifies the Evening rush hours and the count of vehicles per that hour. We can observe that evening rush hours are between 20 -22

#### Calculating the average speed between each junctions on M50


In [75]:
m50cositdf.filter(col("cosit").isin(cositlist)).select(col("cosit"),col("speed")).groupBy("cosit").agg(F.avg("speed").alias("average_speed")).show()


+-----+------------------+
|cosit|     average_speed|
+-----+------------------+
| 1507|102.74182687085913|
| 1500| 93.74959897337183|
| 1506|102.79217719132893|
| 1505| 99.69152287044645|
| 1504|101.99216139028985|
|15010| 105.0165992764418|
|15012|105.10443959243086|
| 1509| 98.35261039422281|
| 1502|102.36304050088046|
|15011|101.79879709487064|
| 1501|101.33084897730457|
| 1503|102.18442775736273|
| 1508| 98.64505637467477|
+-----+------------------+



The above the query gives the average vehicle speed recorded at each counter.

In [100]:
junctionlist =[("Junction 3-4", 1500),("Junction 4-5", 1501),("Junction 5-6", 1502),("Junction 6-7",1508),("Junction 7-9",1503),("Junction 9-10",1509),("Junction 10-11",1504),("Junction 11-12",1505),("Junction 12-13",1506),("Junction 13-14",1507),("Junction 14-15",15010),("Junction 15-16",15011),("Junction 16-17",15012)]
jun= sc.parallelize(junctionlist).collect()
juncdataf=spark.createDataFrame(jun, ["Junctions","cosits"])
m50=m50cositdf.filter(col("cosit").isin(cositlist)).select(col("cosit"),col("speed")).groupBy("cosit").agg(F.round(F.avg("speed"),0).alias("Average Speed"))
Finaldf=m50.alias('a').join(juncdataf.alias('b'),col('a.cosit') == col('b.cosits'))

Finaldata = Finaldf.drop("cosits")
Finaldata.show()

+-----+-------------+--------------+
|cosit|Average Speed|     Junctions|
+-----+-------------+--------------+
| 1505|        100.0|Junction 11-12|
| 1508|         99.0|  Junction 6-7|
| 1501|        101.0|  Junction 4-5|
| 1504|        102.0|Junction 10-11|
|15010|        105.0|Junction 14-15|
| 1506|        103.0|Junction 12-13|
| 1507|        103.0|Junction 13-14|
| 1509|         98.0| Junction 9-10|
| 1500|         94.0|  Junction 3-4|
| 1502|        102.0|  Junction 5-6|
| 1503|        102.0|  Junction 7-9|
|15011|        102.0|Junction 15-16|
|15012|        105.0|Junction 16-17|
+-----+-------------+--------------+



This is a refined output for this question, I have added the  junction numbers at which the respective counter is located and the percentage is rounded to 1 decimal.

#### Calculating the top 10 locations with highest number of counts of HGVs(class)and Mapping  the COSITs with their names given on the map

In [74]:
locationlist =[("TMU M50 001.7 N", 1500),("TMU M50 005.0 N", 1501),("TMU M50 010.0 N", 1502),("TMU M50 015.0 S",1508),("TMU M50 020.0 N",1503),("TMU M50 015.0 N",1509),("TMU M50 025.0 S",1504),("TMU M50 025.0 N",1505),("TMU M50 030.0 S",1506),("TMU M50 035.0 S",1507),("TMU M50 040.0 S",15010),("TMU M50 035.0 N",15011),("TMU M50 040.0 N",15012)]
nm= sc.parallelize(locationlist).collect()
dataf=spark.createDataFrame(nm, ["location","cosits"])
dataf.show()

+---------------+------+
|       location|cosits|
+---------------+------+
|TMU M50 001.7 N|  1500|
|TMU M50 005.0 N|  1501|
|TMU M50 010.0 N|  1502|
|TMU M50 015.0 S|  1508|
|TMU M50 020.0 N|  1503|
|TMU M50 015.0 N|  1509|
|TMU M50 025.0 S|  1504|
|TMU M50 025.0 N|  1505|
|TMU M50 030.0 S|  1506|
|TMU M50 035.0 S|  1507|
|TMU M50 040.0 S| 15010|
|TMU M50 035.0 N| 15011|
|TMU M50 040.0 N| 15012|
+---------------+------+



I have a created a list of location names and its cosits and converted it to a dataframe 

In [108]:
df=m50cositdf.select(m50cositdf.cosit,m50cositdf.classname)


In [83]:
mergeddf=df.alias('a').join(dataf.alias('b'),col('a.cosit') == col('b.cosits'))

mergeddf1 = mergeddf.drop("cosits")

qw=["HGV_RIG","HGV_ART"]
mergeddf1.select(mergeddf1.cosit,mergeddf1.classname,mergeddf1.location).filter(mergeddf1.classname.isin(qw)).groupby("cosit","location").count().sort("count", ascending =False).show(10)

+-----+---------------+-----+
|cosit|       location|count|
+-----+---------------+-----+
| 1508|TMU M50 015.0 S| 1224|
| 1502|TMU M50 010.0 N| 1186|
| 1503|TMU M50 020.0 N|  962|
| 1501|TMU M50 005.0 N|  923|
| 1500|TMU M50 001.7 N|  663|
| 1509|TMU M50 015.0 N|  375|
| 1504|TMU M50 025.0 S|  291|
| 1505|TMU M50 025.0 N|  218|
| 1506|TMU M50 030.0 S|  217|
|15010|TMU M50 040.0 S|  189|
+-----+---------------+-----+
only showing top 10 rows



I have joined the two dataframes to the get the location name next to the cosit, The above query gives the top 10 locations and its cosits with highest counts of vehicles.